In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/songfeng/APP/env/python/anaconda3/envs/tensorflow14/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([

In [2]:
df= pd.read_csv("../data/test_data2.csv")

In [3]:
df.head()

,label_name,features
0,854,"115349,21589,1024,141917,854"
1,14658,"115349,21589,1024,141917,854"
2,854,"21589,1024,141917,854,854"
3,14658,"21589,1024,141917,854,854"
4,94828,"1024,141917,854,854,854"


In [147]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def serialize_example(label, feature):
    """
    Creates a tf.Example message ready to be written to a file.
    """

    # Create a dictionary mapping the feature name to the tf.Example-compatible
    # data type.

    #   feature = {
    #       'label': _int64_feature(label),
    #       'feature': _int64_list_feature(feature)
    #   }
    #   print()  
    a = tf.train.Features(feature={
        "label": tf.train.Feature(int64_list=tf.train.Int64List(value=label))
    })
    feature_list = tf.train.FeatureList(
        feature=[tf.train.Feature(int64_list=tf.train.Int64List(value=[v])) for v in feature])
    print(feature_list)
    b = tf.train.FeatureLists(feature_list={
        "feature": feature_list
    })
    print(b)

    # Create a Features message using tf.train.Example.

    example_proto = tf.train.SequenceExample(context=a, feature_lists=b)
    return example_proto.SerializeToString()

In [148]:
print(type(_int64_list_feature([894])))

<class 'tensorflow.core.example.feature_pb2.Feature'>


In [149]:
serialized_example = serialize_example(label= [94828],feature= [1024, 141917, 854, 854, 854])

feature {
  int64_list {
    value: 1024
  }
}
feature {
  int64_list {
    value: 141917
  }
}
feature {
  int64_list {
    value: 854
  }
}
feature {
  int64_list {
    value: 854
  }
}
feature {
  int64_list {
    value: 854
  }
}

feature_list {
  key: "feature"
  value {
    feature {
      int64_list {
        value: 1024
      }
    }
    feature {
      int64_list {
        value: 141917
      }
    }
    feature {
      int64_list {
        value: 854
      }
    }
    feature {
      int64_list {
        value: 854
      }
    }
    feature {
      int64_list {
        value: 854
      }
    }
  }
}



In [130]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "label"
    value {
      int64_list {
        value: 94828
      }
    }
  }
}

In [131]:
df["feature2"] = df["features"].map(lambda x:[ int(v) for v in x.split(",")])
df["label"] = df["label_name"].map(lambda x: int(x))

In [132]:
df.head()

,label_name,features,feature2,label
0,854,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",854
1,14658,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",14658
2,854,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",854
3,14658,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",14658
4,94828,"1024,141917,854,854,854","[1024, 141917, 854, 854, 854]",94828


In [133]:
filename= "test2.tfrecord"
with tf.python_io.TFRecordWriter(filename) as writer:
  for label,feature in zip(df["label"],df["feature2"]):
    example = serialize_example([label],feature)
    writer.write(example)
    

In [168]:
# Create a description of the features.
context_features = {
            'label':tf.FixedLenFeature([],dtype=tf.int64)
}
sequence_features = {
            "feature":tf.FixedLenSequenceFeature([],dtype=tf.int64,allow_missing=False),

}

def _parse_function(example_proto):
  # Parse the input tf.Example proto using the dictionary above.
  return tf.parse_single_sequence_example(example_proto,context_features=context_features,sequence_features= sequence_features)

In [169]:
x = _parse_function(serialized_example)

In [170]:
dataset = tf.data.TFRecordDataset("test2.tfrecord").map(_parse_function)

In [176]:
iterator= dataset.make_one_shot_iterator()


In [196]:
(label,feature) = iterator.get_next()
with tf.Session() as sess:
    for i in range(10):
        _label,_feature = sess.run([label['label'],feature['feature']])
        print(_feature)
    

[115349  21589   1024 141917    854]
[115349  21589   1024 141917    854]
[ 21589   1024 141917    854    854]
[ 21589   1024 141917    854    854]
[  1024 141917    854    854    854]
[  1024 141917    854    854    854]
[141917    854    854    854  94828]
[141917    854    854    854  94828]
[  854   854   854 94828 14658]
[  854   854   854 94828 14658]


In [186]:
df.head(10)

,label_name,features,feature2,label
0,854,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",854
1,14658,"115349,21589,1024,141917,854","[115349, 21589, 1024, 141917, 854]",14658
2,854,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",854
3,14658,"21589,1024,141917,854,854","[21589, 1024, 141917, 854, 854]",14658
4,94828,"1024,141917,854,854,854","[1024, 141917, 854, 854, 854]",94828
5,14658,"1024,141917,854,854,854","[1024, 141917, 854, 854, 854]",14658
6,14658,"141917,854,854,854,94828","[141917, 854, 854, 854, 94828]",14658
7,14658,"141917,854,854,854,94828","[141917, 854, 854, 854, 94828]",14658
8,14658,"854,854,854,94828,14658","[854, 854, 854, 94828, 14658]",14658
9,14658,"854,854,854,94828,14658","[854, 854, 854, 94828, 14658]",14658


In [187]:
!cp test2.tfrecord /Users/songfeng/workspace/club_factory/my_project/embedding/data